# Two beads on a string

Nous allons dans cet exemple procéder à la démonstration la plus complète de la librairie `quantum-string`

On commence par importer les librairies pertinentes

In [19]:
from simulation import Simulation
from particle import Particle, Particles
from edge import ExcitatorSin, MirrorEdge

import numpy as np

import os

On définit un chemin pour la sauvegarde des simulations. On peut prendre le chemin d'accès contenant ce fichier à l'aide de `os.path` ou alors choisir n'importe quel chemin (existant)

In [20]:
mypath = "C:\\Users\\leog\\Desktop\\lg2021stage\\output"

On va à présent définir les quantités pertinentes pour la simulation. 

On définit la longueur de la corde $ L $, la tension $ T $ en tout point, et la densité de la corde $ \rho_0 $.

Et on choisit le nombre de cellules $ n_x $ (`space_steps`) que composera la corde.

On définit la célérité $ c $, puis notre distance entre chaque point tel que $ n_x \Delta x = L $
puis on choisit notre $ \Delta t $ qui doit respecter $ c = \frac{\Delta x}{\Delta t} $

In [21]:
duration = 4.0 # duration of simulation [s]
length = 5.0 # [m]
tension = 1.0 # [N]
density = 0.01 # [kg/m]
space_steps = 1024 # string discretisation

celerity = np.sqrt(tension/density)

dx = length/(space_steps - 1)
dt = dx/celerity
time_steps = int(duration/dt)

On définit le champ initial et la vitesse initial de la corde. On la représente par une liste où chaque élément correspond à la valeur du champ en la cellule considérée.

Ici on choisit un champ nul ainsi qu'une dérivée nulle. On créé donc une liste contenant uniquement des zéros, de taille `space_steps`

In [22]:
initial_field = [0.0]*space_steps
initial_vel_field = [0.0]*space_steps

A présent nous allons nous occuper des conditions aux bords de la corde. Nous allons utiliser les classes prévues à cet effet de notre module.

Ici, nous allons créer un exitateur sinusoïdal (`ExcitatorSin`) au bout gauche, et un mirroir parfait (`MirrorEdge`) au bord droit. Se référer à la documentation, d'autres conditions existent.

In [23]:
signal_pulsation = 25.0*2.0*np.pi # [rad/s]
edge_left = ExcitatorSin(dt, 0.01, signal_pulsation, 0.0)
edge_right = MirrorEdge()

Puis nous créons les particules individuellement à l'aide de la classe `Particle`. On donne en argument la position initiale de la particule (id de la cellule), sa vitesse initiale, sa masse $ m $, sa pulsation $ \omega = \sqrt{\frac{K}{m}} $, un booléen (`True`) qui indique si elle est fixée sur la corde, et le nombre total de cellules que compose la corde.

Pour finir, on rassemble toutes ces particules dans un objet `Particles`

In [24]:
pos_a = 0.4
cell_a = int(pos_a*space_steps)
particle_a = Particle(cell_a, 0.0, 0.01, 2*np.pi, True, space_steps)

pos_b = 0.6
cell_b = int(pos_b*space_steps)
particle_b = Particle(cell_b, 0.0, 0.02, 2*np.pi, True, space_steps)

particles = Particles(particle_a, particle_b)

On va maintenant initialiser la simulation avec la classe `Simulation`, en rentrant les paramètres que nous venons de définir. On oublie pas de `print` la simulation avant de la lancer, pour être sûr que tous nos paramètres nous conviennent

In [26]:
simu = Simulation(dt, time_steps, space_steps, length, density, tension, edge_left, edge_right, initial_field, initial_vel_field, particles)
print(simu)

[SIMULATION]    Δt=0.00048828125s, Δx=0.0048828125m, time steps=8192, string steps (nb discretisation)=1024
[STRING]    L=5.0m, T=1.0N, rho=0.0kg/m, c=10.0m/s ; 2 particles
[PARTICLES]    ;0: m=0.01kg, omega=6.28rad/s;1: m=0.02kg, omega=6.28rad/s;


On peut maintenant lancer la simulation. Le lancement prend en argument obligatoire le chemin pour sauvegarder la simulation, et un tas d'options (non exhaustif):
* `file=True` génère un fichier texte contenant la valeur des champs à chaque instant dans un fichier, et la position en $ x $ des particules dans un autre (inutile si les particules sont fixes)
* `anim=True` génère une animation vidéo de la simulation
* `compress=True` permet de compresser le fichier .mp4 après sa création
* `frameskip` permet de gérer le frameskip _pour la vidéo._ Si `frameskip=1`, toutes les frames sont enregistrées dans la vidéo. Si `frameskip=n` 1 frame sur `n` seront gardées.
* `window_anim` permet de fenêtrer l'animation, pour se focaliser sur une zone qui nous intéresse. Si `window_anim=False` toute la corde sera représentée (équivalent à `(0.0, 1.0)`). Sinon, on met un tuple qui correspond à la fenêtre souhaitée (ex: `(0.0, 0.5)`)
* `yscale` permet de rescale l'axe $ y $ de la corde. Par défaut, `yscale=5.0` (zoomé fois 5) 

Une fois la simulation terminée, la fonction `Simulation.run` renvoie le chemin des deux fichiers champ + particules (si créés)

In [27]:
field_path, _ = simu.run(mypath, anim=True, file=True, compress=False)

[SIMULATION]    Δt=0.00048828125s, Δx=0.0048828125m, time steps=8192, string steps (nb discretisation)=1024
[STRING]    L=5.0m, T=1.0N, rho=0.0kg/m, c=10.0m/s ; 2 particles
[PARTICLES]    ;0: m=0.01kg, omega=6.28rad/s;1: m=0.02kg, omega=6.28rad/s;

video output creation...
video created successfully!


On peut maintenant procéder à une FFT sur les zones du champ pertinentes. Dans cet exemple, on peut faire une FFT avant la première masse, après la seconde masse, et une entre les deux. On va simplement utiliser la class `PostProcess` et sa fonction `fourier` qui va permettre de procéder à cette FFT. On lui donne en argument les fenêtres considérées 

In [ ]:
field_file = open(field_path, "r") # open the filed simulation file

win1 = (0.0, 0.25)
win2 = (0.35, 0.65)
win3 = (0.75, 1.0)

pp = PostProcess(field_file, log=True)
pp.fourier(win1, win2, win3, frameskip=20, path=mypath) # makes the fourier analysis on the windows given

Pour chaque fenêtre, on aura une animation de l'évolution de la FFT au fil du temps dans la fenêtre, et un spectrographe